In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import *
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import copy
from bokeh.models import (ColumnDataSource, LogAxis, LinearAxis, Rect, FactorRange,
                          CategoricalAxis, Line, Text, Square, HoverTool)
from bokeh.models import Plot, Range1d, ImageURL, DataRange1d, Select
from bokeh.embed import components
from bokeh.models import CustomJS, Slider
from bokeh.io import curdoc
from bokeh.plotting import figure, hplot, vplot, output_file, show
from bokeh.layouts import row, widgetbox

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)
consump = Consumption()

policy_T = Policy()
records_reform = Records(puf)
consump_T = Consumption()

policy_H = Policy()
records_org = Records(puf)
consump_H = Consumption()

calcbase = Calculator(policy = policy_base, records = records_base,consumption=consump)
calc_T = Calculator(policy = policy_T, records = records_reform,consumption=consump_T)
calc_H = Calculator(policy = policy_H, records = records_org,consumption=consump_H)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform_H= {
    2017: {'_AGI_surtax_thd': [[5000000, 5000000, 5000000, 5000000, 5000000, 5000000]],
           '_AGI_surtax_trt': [0.04],
           '_FST_AGI_trt':[0.3],
           '_ID_BenefitCap_rt': [0.28]
        }}

In [4]:
reform_T = {
    2017: {'_II_rt1': [0.12],
           '_II_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
           '_II_rt2': [0.25],
           '_II_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_II_rt3': [0.25],
           '_II_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_II_rt4': [0.25],
           '_II_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_II_rt5': [0.25],
           '_II_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_II_rt6': [0.25],
           '_II_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_II_rt7': [0.33],
           '_PT_rt1': [0.12],
           '_PT_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
           '_PT_rt2': [0.25],
           '_PT_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_PT_rt3': [0.25],
           '_PT_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_PT_rt4': [0.25],
           '_PT_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_PT_rt5': [0.25],
           '_PT_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_PT_rt6': [0.25],
           '_PT_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_PT_rt7': [0.33],
           '_CG_thd1': [[37500, 75000, 37500, 37500, 75000, 37500]],
           '_CG_thd2': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_AMT_CG_thd1': [[37500, 75000, 37500, 37500, 75000, 37500]],
           '_AMT_CG_thd2': [[112500, 225000, 112500, 112500, 225000, 112500]],
           '_AMT_trt1':[0],
           '_AMT_trt2':[0],
           '_NIIT_trt':[0],
           '_STD': [[15000,30000,15000,15000,12600, 6300, 1050]],
           '_II_em': [0],
           '_ID_BenefitSurtax_crt': [0.0],
           '_ID_BenefitSurtax_trt': [1.0],
           '_ID_BenefitSurtax_em': [[100000, 200000, 100000, 100000, 200000, 100000]]
        }}

In [5]:
policy_T.implement_reform(reform_T)
policy_H.implement_reform(reform_H)
calcbase.advance_to_year(2017)
calc_T.advance_to_year(2017)
calc_H.advance_to_year(2017)

In [6]:
calcbase.calc_all()
calc_T.calc_all()
calc_H.calc_all()

In [9]:
def source_data(data_1, data_2):
    data_1.index = (data_1.reset_index()).index
    data_2.index = (data_2.reset_index()).index
    df = data_1
    df['reform_2'] = data_2['reform']
    return ColumnDataSource(df)

In [8]:
def make_a_plot(source):
    plot = figure(plot_height=600, plot_width=800,
                  tools="")
    plot.line('index','reform',source = source, line_color=Greens4[0], line_width=2, line_alpha=.8)
    plot.line('index','reform_2',source = source, line_color=Reds4[0], line_width=2, line_alpha=.8)
    plot.line('index','base',source = source, line_color=Blues4[0], line_width=2, line_alpha=1.2)
    plot.yaxis.axis_label = 'Avg. MTR on Wage Income'
    plot.xaxis.axis_label = 'Percentile of Adjusted Gross Income'
    return plot

In [14]:
source1 =get_mtr_data(calcbase,calc_T,MARS = 1,weights = wage_weighted, income_measure = 'c00100', complex_weight = False)
source2 =get_mtr_data(calcbase,calc_H,MARS = 1,weights = wage_weighted, income_measure = 'c00100', complex_weight = False)
source = source_data(source1,source2)
plot = make_a_plot(source)

In [15]:
def update(attr, old, new):
    string = MARS_Select.value
    if string == 'Single':
        id_ = 0
    elif string == 'Joint':
        id_ = 1
    else:
        id_ = 4
    new1 =get_mtr_data(calcbase,calc_T,MARS = id_,weights = wage_weighted, income_measure = 'c00100', complex_weight = False)
    new2 =get_mtr_data(calcbase,calc_H,MARS = id_,weights = wage_weighted, income_measure = 'c00100', complex_weight = False)
    src = source_data(new1,new2)
    source.data.update(src.data)

In [16]:
mars = 'Single'
MARS_Select = Select(value=mars, title='MARS', options=['Single', 'Joint', 'Head of Household'])
MARS_Select.on_change('value',update)

In [17]:
layout = row(
    plot,
    MARS_Select
)
show(layout)